In [1]:
#####################
#     LOAD DATA     #
#####################

import json_lines
import csv

def process_tweet(tweet):  
    d = {}
    d['hashtags'] = [hashtag['text'] for hashtag in tweet['entities']['hashtags']]
    d['text'] = tweet['full_text']
    d['user'] = tweet['user']['screen_name']
    d['user_loc'] = tweet['user']['location']
    d['created_at'] = tweet['created_at']
    return d

if False:
    with open('congress_dataset/senators-1.jsonl', 'rb') as f:
        with open(r'senators-1-tweets.csv', 'a') as file:
            writer = csv.writer(file)
            for item in json_lines.reader(f):
                # Only collect tweets in English
                if item['lang'] == 'en' and len(item['entities']['hashtags']) > 0:
                    tweet_data = process_tweet(item)
                    writer.writerow(list(tweet_data.values()))

                    
import pandas as pd
tweets = pd.read_csv("senators-1-tweets.csv", header=None, names=['hashtags', 'text', 'user', 'user_location', 'created_at'])  
print('num tweets: {}'.format(len(tweets)))


import spacy
nlp = spacy.load('en_core_web_md')

import random

docs = []
N = 50000
rand_tweets = random.sample(range(len(tweets)), k=N)
for i, tw in enumerate(rand_tweets):
    if i % 1000 == 0:
        print('{}%'.format(100./N*i), end=' ')
    docs.append(nlp(tweets.iloc[i]['text']))
    
stop_words = ['senator', '\'s', '-PRON-', '’', '’s', 'amp', 'i' ]


# Clean tweets here
cleaned_tweets = []
cleaned_tweets_text = []
for doc in docs:
    curr_tweet = []
    for w in doc:
        if w.text == '\n' or w.is_stop or w.is_punct or w.like_num or w.like_url: continue
        if w.is_space or w.is_oov or not w.is_ascii or w.text.lower() in stop_words: continue
        curr_tweet.append(w.text.lower()) # add lemmatized version of the word
    if curr_tweet:
        cleaned_tweets.append(curr_tweet)
        cleaned_tweets_text.append(' '.join(curr_tweet))
        
        


num tweets: 449334
0.0% 2.0% 4.0% 6.0% 8.0% 10.0% 12.0% 14.0% 16.0% 18.0% 20.0% 22.0% 24.0% 26.0% 28.0% 30.0% 32.0% 34.0% 36.0% 38.0% 40.0% 42.0% 44.0% 46.0% 48.0% 50.0% 52.0% 54.0% 56.0% 58.0% 60.0% 62.0% 64.0% 66.0% 68.0% 70.0% 72.0% 74.0% 76.0% 78.0% 80.0% 82.0% 84.0% 86.0% 88.0% 90.0% 92.0% 94.0% 96.0% 98.0% 

In [119]:
# here we should get only cleaned tweets
import gensim
from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
from gensim.models.wrappers import LdaMallet
from gensim.corpora import Dictionary

N_SMALL = 2000

cleaned_tweetz = cleaned_tweets[:N_SMALL]

bigram = gensim.models.Phrases(cleaned_tweetz)
cleaned_tweetz = [bigram[t] for t in cleaned_tweetz]

# create dictionary and corpus
dictionary = Dictionary(cleaned_tweetz)

# corpus = (token_id, count_in_curr_doc) , sparse representation
corpus = [dictionary.doc2bow(clean_tween) for clean_tween in cleaned_tweetz]
cleaned_tweetz_id = [dictionary.doc2idx(document=tw) for tw in cleaned_tweetz]

In [120]:
import pyro
from torch.distributions import constraints
from pyro import distributions as p

from pyro import optim, infer
import torch

K = 10
D = len(cleaned_tweetz_id)
V = len(dictionary)

def lda_naive_model(docs):
    
    alpha1 = torch.ones(K)
    
    alpha2 = torch.ones(V)
    
    # distributions over words in a topic / for each topic
    
    word_distrib = pyro.sample('phi', p.Dirichlet(alpha2).expand([K]).independent(1))

    # for each doc
    for i in pyro.plate('data_loop', len(docs)):
        doc = docs[i]
        # topic distribution per document
        theta = pyro.sample('theta_{}'.format(i), p.Dirichlet(alpha1))
        
        # topic of each document - sample topic
        this_topic = pyro.sample('topic_{}'.format(i), p.Categorical(theta))
        
        # for each doc sample each word 
        words = pyro.sample('words_{}'.format(i), 
                            p.Categorical(word_distrib[this_topic]).expand([len(doc)]).independent(1), obs=doc)            
            
def lda_naive_guide(docs):
    alpha1_q = pyro.param('alpha1_q', torch.ones(K), constraint=constraints.positive)
    alpha2_q = pyro.param('alpha2_q', torch.ones(V), constraint=constraints.positive)
    
    word_distrib = pyro.sample('phi', p.Dirichlet(alpha2_q).expand([K]).independent(1))
    
    for i in pyro.plate('data_loop', len(docs)):
        theta = pyro.sample('theta_{}'.format(i), p.Dirichlet(alpha1_q))
        pyro.sample('topic_{}'.format(i), p.Categorical(theta))
        
    return theta, word_distrib
        
    
#pyro.clear_param_store()
    
# set up the optimizer
adam_params = {"lr": 1e-2}
optimizer = optim.Adam(adam_params)

# setup the inference algorithm
svi = infer.SVI(lda_naive_model, lda_naive_guide, optimizer, loss=infer.Trace_ELBO())

docs_batch = [torch.Tensor(tw) for tw in cleaned_tweetz_id]

epochs = 100
for epoch in range(epochs):
    epoch_loss = 0
    
    loss = svi.step(docs_batch)
    print(loss)
    
global_topic_distr, words_per_topic_distr = lda_naive_guide(cleaned_tweetz_id[:2])


379106.9965648651
380639.26475286484
379892.0476179123
376195.62323999405
378332.0884270668
373684.2842683792
376006.81881427765
374135.1972284317
373812.3332004547
373366.8222732544
375923.7884426117
373673.2307167053
372357.4429092407
372229.59427547455
370778.23274707794
369433.6941947937
371287.7541847229
372340.01037693024
368826.22997379303
370993.00379276276
370496.2372045517
368216.4705314636
370547.87199926376
368496.6466636658
368994.9569554329
367078.02571964264
368306.80543518066
366265.35385274887
367748.13159513474
363994.3721933365
365032.19675540924
365924.99113845825
363877.7126612663
364387.7191786766
363806.77124786377
364299.6677389145
364659.393491745
364666.11958503723
362673.06474637985
362138.34634542465
361261.0774798393
364710.36989307404
360317.04695415497
361712.978808403
361359.87417411804
357400.7515897751
361951.12527513504
360037.71829891205
360479.4944868088
360611.9307150841
358484.11059331894
358905.84768533707
360047.48643398285
357923.1809258461
358

In [91]:
global_topic_distr

tensor([0.0063, 0.0939, 0.0538, 0.2134, 0.0840, 0.1333, 0.0017, 0.1731, 0.0948,
        0.1457], grad_fn=<_DirichletBackward>)

In [92]:
for t in range(K):
    print("---- topic {} -----".format(t))
    top5_words = (torch.argsort(words_per_topic_distr[t])[:5]).cpu().numpy()
    top5_words = list(map(lambda x: dictionary[x], top5_words))
    print(top5_words)
    

    


---- topic 0 -----
['rise', 'governor', 'despite', 'provides', 'prime']
---- topic 1 -----
['someone', 'tragic', 'familiarize', 'charity', 'ultimate']
---- topic 2 -----
['records', 'etowah', 'disrupt', 'uab', 'debt']
---- topic 3 -----
['corporations', 'certainly', 'civic', '19th', 'guns']
---- topic 4 -----
['myths', 'heritage', 'valid', 'pony', 'during']
---- topic 5 -----
['hours', 'opponent', 'foreign', 'street', 'willing']
---- topic 6 -----
['narragansett', 'organizations', 'each', 'heal', 'greeting']
---- topic 7 -----
['est', 'judge', 'affront', 'tackling', 'tbi']
---- topic 8 -----
['seated', 'uphill', 'agency', 'ne', 'required']
---- topic 9 -----
['may', 'roads', 'roy', 'inventions', 'coffee']


In [36]:
###### LDA

K = 10
D = len(cleaned_tweetz_id)
V = len(dictionary)

@pyro.poutine.broadcast
def model_lda(docs):
    phi = pyro.sample('phi', p.Dirichlet(torch.ones([K, V])).independent(1))
    
    for d in pyro.irange('documents', len(docs)):
        theta_d = pyro.sample(f"theta_{d}", p.Dirichlet(torch.ones([K])))
        
        with pyro.iarange(f"words_{d}", len(docs[d])):
            topic = pyro.sample(f"topic_{d}", p.Categorical(theta_d))
            pyro.sample(f"w_{d}", p.Categorical(phi[topic]), obs=docs[d])

@pyro.poutine.broadcast         
def guide(docs):
    beta_q = pyro.param("beta_q", torch.ones([K,V]), constraint=constraints.positive)
    phi_q = pyro.sample("phi", p.Dirichlet(beta_q).independent(1))
    
    for d in pyro.irange("document", len(docs)):
        alpha_q = pyro.param(f"alpha_q_{d}", torch.ones([K]), constraint = constraints.positive)
        q_theta_d = pyro.sample(f"theta_{d}", p.Dirichlet(alpha_q))
        
        with pyro.iarange(f"words_{d}", len(docs[d])):
            q_i = pyro.param(f"q_{d}", torch.randn([len(docs[d]), K]).exp(), constraints.simplex)
            pyro.sample(f"topic_{d}", p.Categorical(q_i))
            
pyro.clear_param_store()  

adam_params = {"lr": 0.01, "betas": (0.90, 0.999)}
optimizer = optim.Adam(adam_params)

svi = infer.SVI(model_lda, infer.config_enumerate(guide, 'parallel'), 
                optimizer, loss=infer.TraceEnum_ELBO(max_iarange_nesting=1))

epochs = 1
for epoch in range(epochs):
    epoch_loss = 0
    n_batches = len(cleaned_tweetz_id)
    #print(list(pyro.get_param_store().items()))
    for i in range(1):
        docs_batch = [torch.Tensor(tw) for tw in cleaned_tweetz_id[i*32:i*32 + 128]]
        counter = 0
        loss = svi.step(docs_batch)
        epoch_loss += loss
    
        
    print(counter)
    print("Epoch {} | Avg. loss: {} |  N_batches: {}".format(epoch, epoch_loss / n_batches, counter))


RuntimeError: bitset::set: __position (which is 64) >= _Nb (which is 64)
 Trace Shapes:                                                                                                                                                                                                           
  Param Sites:                                                                                                                                                                                                           
 Sample Sites:                                                                                                                                                                                                           
      phi dist                                                                                                                                                                                                  | 10 2388
         value                                         

In [12]:
b = pyro.sample('test', p.Dirichlet(torch.ones(5)))
pyro.sample('test2', p.Categorical(b))

tensor(1)